Classify:

In [ ]:
.\.venv\Scripts\activate
python -m speciesnet.scripts.run_model --folders ".\test_images\" --predictions_json "results/output_speciesnet_pl.json" --country POL

Import data

In [2]:
import json
import pandas as pd

with open('results/output_speciesnet_pl.json', 'r') as file:
    data = json.load(file)

Parse data

In [3]:
output = {}
image_names = []
classified_animals = []

for d in data['predictions']:
    file_name = d['filepath'].split(sep='/')[-1]
    image_names.append(file_name)

    species = {}

    for s in zip(d['classifications']['classes'], d['classifications']['scores']):
        species_name = s[0].split(sep=';')[-1]
        species.setdefault(species_name, s[1])

    classified_animals.append(d['classifications']['classes'][0].split(sep=';')[-1])

    #print(file_name, classifications, scores)
    output.setdefault(file_name, species)

with open('results/output_speciesnet_pl_cleared.json', 'w') as file:
    json.dump(output, file, indent=2)

df = pd.DataFrame({'image': image_names, 'detected_animal': classified_animals})
df.to_csv('results/results_speciesnet.csv')
df

,image,detected_animal
0,2020-01-07 00-47-51.JPG,common fallow deer
1,2022-09-05 13-57-18.JPG,red fox
2,2022-09-06 19-34-11.JPG,blank
3,2022-09-10 22-41-29.JPG,eurasian badger
4,2022-09-13 01-29-08.JPG,martes species
...,...,...
116,IMG_0607.JPG,american bison
117,IMG_0726.JPG,coyote
118,IMG_0800.JPG,domestic cat
119,IMG_0814.JPG,virginia opossum


Parse POL

In [4]:
image_names = []
classified_animals = []

for d in data['predictions']:
    file_name = d['filepath'].split(sep='/')[-1]
    image_names.append(file_name)
    predicted_animal = d['prediction'].split(sep=';')[-1]
    classified_animals.append(predicted_animal)

df = pd.DataFrame({'image': image_names, 'detected_animal': classified_animals})
df.to_csv('results/results_speciesnet_pl.csv')
df

,image,detected_animal
0,2020-01-07 00-47-51.JPG,common fallow deer
1,2022-09-05 13-57-18.JPG,red fox
2,2022-09-06 19-34-11.JPG,blank
3,2022-09-10 22-41-29.JPG,eurasian badger
4,2022-09-13 01-29-08.JPG,martes species
...,...,...
116,IMG_0607.JPG,bovidae family
117,IMG_0726.JPG,canis species
118,IMG_0800.JPG,felis species
119,IMG_0814.JPG,mammal
